In [4]:
import pandas as pd
from sklearn.metrics import precision_recall_fscore_support
from collections import Counter

In [5]:
data = pd.read_csv('car_evaluation_data.csv')
data
#data.head()

,buying,maint,doors,persons,lug_boot,safety,outcome
0,vhigh,vhigh,2,2,small,low,unacc
1,vhigh,vhigh,2,2,small,med,unacc
2,vhigh,vhigh,2,2,small,high,unacc
3,vhigh,vhigh,2,2,med,low,unacc
4,vhigh,vhigh,2,2,med,med,unacc
...,...,...,...,...,...,...,...
1723,low,low,5,5,med,med,good
1724,low,low,5,5,med,high,vgood
1725,low,low,5,5,big,low,unacc
1726,low,low,5,5,big,med,good


In [6]:
data.shape

(1728, 7)

In [8]:
data.outcome.value_counts()

unacc    1210
acc       384
good       69
vgood      65
Name: outcome, dtype: int64

In [20]:
X = data.iloc[:,:-1]
y = data.outcome
X.head()

,buying,maint,doors,persons,lug_boot,safety
0,vhigh,vhigh,2,2,small,low
1,vhigh,vhigh,2,2,small,med
2,vhigh,vhigh,2,2,small,high
3,vhigh,vhigh,2,2,med,low
4,vhigh,vhigh,2,2,med,med


In [24]:
from sklearn.preprocessing import LabelEncoder
enc = LabelEncoder()
X.persons.value_counts
X.loc[:,['buying','maint','lug_boot','safety']] = X.loc[:,['buying','maint','lug_boot','safety']].apply(enc.fit_transform)
#X.loc[:,['buying','maint','lug_boot','safety', 'persons']] = \
#X.loc[:,['buying','maint','lug_boot','safety', 'persons']].apply(enc.fit_transform)

X.head()
#X.persons.value_counts

,buying,maint,doors,persons,lug_boot,safety
0,3,3,2,2,2,1
1,3,3,2,2,2,2
2,3,3,2,2,2,0
3,3,3,2,2,1,1
4,3,3,2,2,1,2


In [25]:
from sklearn.model_selection import train_test_split
X_train,X_test, y_train,y_test = \
train_test_split(X,y,test_size=0.3,random_state=10)

In [26]:
from sklearn.neighbors import KNeighborsClassifier
model = KNeighborsClassifier()
model.fit(X_train,y_train)
y_predict = model.predict(X_test)

In [27]:
from sklearn.metrics import accuracy_score
acc1=accuracy_score(y_test,y_predict)
print(acc1)
pd.crosstab(y_test,y_predict)

0.928709055876686


col_0,acc,good,unacc,vgood
outcome,,,,
acc,89,2,11,0
good,10,10,1,0
unacc,5,0,366,0
vgood,5,2,1,17


In [10]:
#Acccuracy for vgood  out of 25

In [11]:
17/25

0.68

In [12]:
#calculate precision_recall_fscore_support

In [28]:
eval1=precision_recall_fscore_support(y_test,y_predict, average='macro')

In [29]:
eval1

(0.874124671049219, 0.7538156017123832, 0.8001385691717444, None)

In [30]:
type(eval1)

tuple

In [31]:
eval1[0]

0.874124671049219

In [33]:
cols = ['DataType', 'Accuracy Score', 'Precision Score', 'Recall Score','f1-score','Support']

# creating an empty dataframe of the colums
result_tabulation = pd.DataFrame(columns = cols)

# compiling the required information
eval1dict = pd.Series({'DataType': "No-SMOTE",
                       'Accuracy Score': acc1,
                 'Precision Score': eval1[0],      
                    'Recall Score': eval1[1],
                     'f1-score':eval1[2],
                      'Support':eval1[3]})




# appending our result table
result_tabulation = result_tabulation.append(eval1dict , ignore_index = True)

result_tabulation

,DataType,Accuracy Score,Precision Score,Recall Score,f1-score,Support
0,No-SMOTE,0.928709,0.874125,0.753816,0.800139,None


# Now use SMOTE to increase the data

In [34]:
#!pip install imblearn
#or use CONDA install 
#conda install -c conda-forge imbalanced-learn

In [35]:
from imblearn.over_sampling import SMOTE
smote = SMOTE()

In [36]:
X_train_smote, y_train_smote = smote.fit_sample(X_train.astype('float'),y_train)

In [37]:

print("Before SMOTE :" , Counter(y_train))
print("After SMOTE :" , Counter(y_train_smote))

Before SMOTE : Counter({'unacc': 839, 'acc': 282, 'good': 48, 'vgood': 40})
After SMOTE : Counter({'acc': 839, 'unacc': 839, 'vgood': 839, 'good': 839})


In [38]:
y_train_smote.shape

(3356,)

In [39]:
model.fit(X_train_smote,y_train_smote)
y_predict = model.predict(X_test)

acc2=accuracy_score(y_test,y_predict)
print(acc2)
pd.crosstab(y_test,y_predict)


0.9364161849710982


col_0,acc,good,unacc,vgood
outcome,,,,
acc,89,8,3,2
good,1,20,0,0
unacc,16,1,353,1
vgood,0,1,0,24


In [41]:
24/25

0.96

In [42]:
eval2=precision_recall_fscore_support(y_test,y_predict, average='macro')
eval2

(0.8466878076932137, 0.9341031129432905, 0.8835685117873199, None)

In [43]:
# compiling the required information
eval2dict = pd.Series({'DataType': "SMOTE",
                       'Accuracy Score': acc2,
                 'Precision Score': eval2[0],      
                    'Recall Score': eval2[1],
                     'f1-score':eval2[2],
                      'support':eval2[3]})



# appending our result table
result_tabulation = result_tabulation.append(eval2dict , ignore_index = True)

result_tabulation

,DataType,Accuracy Score,Precision Score,Recall Score,f1-score,Support,support
0,No-SMOTE,0.928709,0.874125,0.753816,0.800139,None,NaN
1,SMOTE,0.936416,0.846688,0.934103,0.883569,NaN,None
